# Verse Interpreter development

Essential installs:
- `pip3 install .....`


This version is used to test the first steps for the verse interpreter. The final version will be used as a full python file.

In [1]:
import string
import math
from enum import Enum

## Setting Token Enum for testing purpose

In [2]:
class TokenTypes(Enum):
    # Data
    INTEGER = int

    # Aritmetics
    PLUS = "+"
    MINUS = "-"
    MULTIPLY = "*"
    DIVIDE = "/"

    # Mehtods
    FOR = "for"
    LBRACKET = "("
    RBRACKET = ")"

    # Else
    EOF = None

In [25]:
list(TokenTypes)

[<TokenTypes.INTEGER: <class 'int'>>,
 <TokenTypes.PLUS: '+'>,
 <TokenTypes.MINUS: '-'>,
 <TokenTypes.MULTIPLY: '*'>,
 <TokenTypes.DIVIDE: '/'>,
 <TokenTypes.FOR: 'for'>,
 <TokenTypes.LBRACKET: '('>,
 <TokenTypes.RBRACKET: ')'>,
 <TokenTypes.EOF: None>]

In [3]:
class Token:
    def __init__(self, type: TokenTypes, value) -> None:
        self.type = type
        self.value = value

## Lexer

In [11]:
class lexicon:
    def __init__(self, input: string):
        self.input = input
        self.index = 0
        self.current_char = self.input[self.index]
    
    # moves the pointer a character forward
    def forward(self) -> None:
        self.index += 1

        # checks if index is out of range
        if (self.index >= len(self.input)):
            self.current_char = None
            return
        
        self.current_char = self.input[self.index]
    
    def backward(self) -> None:
        self.index -= 1

        # checks if index is out of range
        if self.index < 0:
            self.current_char = None
            return
        
        self.current_char = self.input[self.index]
    
    def get_int(self) -> int:
        if self.index >= len(self.input):
            return None
        
        result = self.input[self.index]

        # checks if there are multiple digits
        while True:
            self.forward()

            if self.index < len(self.input) and self.input[self.index] != None and self.input[self.index].isnumeric():
                result += self.input[self.index]
            else:
                self.backward()
                break

        return int(result)
    
    def get_var(self) -> string:
        if self.index >= len(self.input):
            return None
        
        result = self.input[self.index]

        # checks if there is a longer variable name
        while True:
            self.forward()

            if self.index < len(self.input) and self.input[self.index] != None and self.input[self.index].isalpha():
                result += self.input[self.index]
            else:
                self.backward()
                break
        
        return result
    
    def get_token(self, char: string) -> Token:
        token = self.check_for_tokentypes(char)

        if token.type != TokenTypes.EOF:
            return token
        
        if char == None:
            return token
            
        # skip spaces.
        if char == ' ':
            self.forward()
            return self.get_token(self.current_char)

        # checks if the current character is a number.
        if char.isnumeric():
            result = self.get_int()
            return Token(TokenTypes.INTEGER, result)
        
        if char.isalpha():
            result = self.get_var()
            token = self.check_for_tokentypes(result)
                  
        return token

    def check_for_tokentypes(self, char: string) -> Token:
         # checks if the current character is a supported token type.
        match char:
            case TokenTypes.DIVIDE.value:
                return Token(TokenTypes.DIVIDE, TokenTypes.DIVIDE.value)
            case TokenTypes.MULTIPLY.value:
                return Token(TokenTypes.MULTIPLY, TokenTypes.MULTIPLY.value)
            case TokenTypes.PLUS.value:
                return Token(TokenTypes.PLUS, TokenTypes.PLUS.value)
            case TokenTypes.MINUS.value:
                return Token(TokenTypes.MINUS, TokenTypes.MINUS.value)
            case TokenTypes.LBRACKET.value:
                return Token(TokenTypes.LBRACKET, TokenTypes.LBRACKET.value)
            case TokenTypes.RBRACKET.value:
                return Token(TokenTypes.RBRACKET, TokenTypes.RBRACKET.value)
            case TokenTypes.FOR.value:
                return Token(TokenTypes.FOR, TokenTypes.FOR.value) 
            case _:
                return Token(TokenTypes.EOF, None)

## Testing Lexer Types

In [5]:
lexer = lexicon("for 123 + 5 * 23")

while lexer.current_char is not None:
    token = lexer.get_token(lexer.current_char)
    print(str(token.value) + " is of the tokentype: " + str(token.type))
    lexer.forward()

for is of the tokentype: TokenTypes.FOR
123 is of the tokentype: TokenTypes.INTEGER
+ is of the tokentype: TokenTypes.PLUS
5 is of the tokentype: TokenTypes.INTEGER
* is of the tokentype: TokenTypes.MULTIPLY
23 is of the tokentype: TokenTypes.INTEGER


## Syntax Tree Nodes

In [6]:
class BaseNode:
    def __init__(self, token) -> None:
        self.token = token
    
    def visit(self, node):
        if isinstance(node, OperatorNode):
            return self.visit_operatorNode(node)
        
        return self.visit_numberNode(node)
        
    def visit_operatorNode(self, node):
        match node.token.type:
            case TokenTypes.DIVIDE:
                return self.visit(node.leftNode) // self.visit(node.rightNode)
            case TokenTypes.MULTIPLY:
                return self.visit(node.leftNode) * self.visit(node.rightNode)
            case TokenTypes.PLUS:
                return self.visit(node.leftNode) + self.visit(node.rightNode)
            case TokenTypes.MINUS:
                return self.visit(node.leftNode) - self.visit(node.rightNode)   

    def visit_numberNode(self, node):
        return node.value

class NumberNode(BaseNode):
    def __init__(self, token) -> None:
        super().__init__(token)
        self.value = token.value

class OperatorNode(BaseNode):
    def __init__(self, token, leftNode: BaseNode, rightNode: BaseNode) -> None:
        super().__init__(token)
        self.leftNode = leftNode
        self.rightNode = rightNode

## Parser

In [7]:
class Parser:
    def __init__(self, lexer: lexicon) -> None:
        self.lexer = lexer
        self.current_token = lexer.get_token(self.lexer.current_char)
    
    def forward(self) -> None:
        self.lexer.forward()
        self.current_token = lexer.get_token(self.lexer.current_char)
    
    def expression(self) -> BaseNode:
        leftNode = self.term()
        token = self.current_token

        if token.type == TokenTypes.PLUS or token.type == TokenTypes.MINUS:
            self.forward()
        
            rightNode = self.term()
            return OperatorNode(token, leftNode, rightNode)
        
        return leftNode

    def term(self) -> BaseNode:
        leftNode = self.number()
        token = self.current_token

        if token.type == TokenTypes.MULTIPLY or token.type == TokenTypes.DIVIDE:
            self.forward()
        
            rightNode = self.number()
            return OperatorNode(token, leftNode,rightNode)
        
        return leftNode
    

    def number(self) -> BaseNode:
        token = self.current_token

        if token.type == TokenTypes.INTEGER:
            self.forward()
            return NumberNode(token)
        
        if token.type == TokenTypes.LBRACKET:
            self.forward()
            node = self.expression()
            self.forward()
            return node
        
        return token
    
    def parse(self) -> BaseNode:
        return self.expression()

In [8]:
class Interpreter:
    def __init__(self, parser: Parser):
        self.parser = parser

    def interpret(self):
        tree = self.parser.parse()
        return tree.visit(tree)

In [12]:
text = "(2 * 3 + 2) * (2 + 4)"
lexer = lexicon(text)
parser = Parser(lexer)
interpreter = Interpreter(parser)
result = interpreter.interpret()
result

48